# Notebook to edit a device config
___To use this notebook...___
1. configure your username and password
2. configure IP or hostname of the device you want to configure
3. If you want to see some more DEBUG messages set loglevel to DEBUG (optional)

### define your login and set backup Directory

In [1]:
username = "lab"
password = "lab"

### on which device do you want to change the configuration?

In [2]:
device = "192.168.0.1"

### set loglevel

In [3]:
loglevel = "INFO"

### The directory where we store the configurations

In [4]:
config_directory = "./device_configs"

# Basic setup .. import, logging and so on

In [5]:
import sys
import difflib
import os
from loguru import logger
from veritas.devicemanagement import napalm as dm
from IPython.display import display, HTML

In [6]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

1

# Connect to device
Change platform, manufacturer and ssh port if neede

In [7]:
conn = dm.Devicemanagement(ip=device,
                           platform="ios",
                           manufacturer="cisco",
                           username=username,
                           password=password,
                           port=22)
conn.open()

# get running-config

In [ ]:
running_config = conn.get_config('running')
running_config = running_config.replace('^C', "\x03")

## If you want to see the config run the next cell

In [ ]:
print(running_config)

## Write config to disk

In [8]:
config_file = f'./{config_directory}/{device}.config'

In [ ]:
directory = os.path.dirname(config_file)
if not os.path.exists(directory):
    logger.debug(f'creating {directory}')
    os.makedirs(directory)
with open(config_file, 'w') as f:
    f.write(running_config)

# Now open config in new tab and edit it. Don't forget to save it after you did your modifications.

# Update config

## load config before we replace it

In [ ]:
with open(config_file, 'r') as f:
   new_config = f.read()

### check if config is okay (optional)

In [ ]:
print(new_config)

## prepare session and upload config
replace=True means that the complete config will be replaced. ___If you only have a part of the configuration, you have to set the parameter to False.___

In [9]:
conn.load_config(config=config_file)

### show diff between running and your new config (optional)

In [ ]:
diff_result = conn.diff_config()
print(diff_result.device_diff)

## Everything correct? Than commit the config change

In [10]:
conn.commit_config()

ReplaceConfigException: Candidate config could not be applied
  

%The input file is not a valid config file.


## otherwise abort the config change

In [ ]:
conn.abort_config()

## clean up

In [ ]:
conn.cleanup()